In [1]:
import numpy as np
import pandas as pd
import os
import textwrap
import sklearn.preprocessing
import sklearn.pipeline
import sklearn.linear_model
import sklearn.model_selection
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, roc_auc_score

In [3]:
data_dir = 'data_readinglevel'
x_train_df = pd.read_csv(os.path.join(data_dir, 'x_train.csv'))
y_train_df = pd.read_csv(os.path.join(data_dir, 'y_train.csv'))

N, n_cols = x_train_df.shape
print("Shape of x_train_df: (%d, %d)" % (N, n_cols))
print("Shape of y_train_df: %s" % str(y_train_df.shape))
print(x_train_df.columns)

Shape of x_train_df: (5557, 32)
Shape of y_train_df: (5557, 5)
Index(['author', 'title', 'passage_id', 'text', 'char_count', 'word_count',
       'sentence_count', 'avg_word_length', 'avg_sentence_length',
       'type_token_ratio', 'pronoun_freq', 'function_words_count',
       'punctuation_frequency', 'sentiment_polarity', 'sentiment_subjectivity',
       'readability_Kincaid', 'readability_ARI', 'readability_Coleman-Liau',
       'readability_FleschReadingEase', 'readability_GunningFogIndex',
       'readability_LIX', 'readability_SMOGIndex', 'readability_RIX',
       'readability_DaleChallIndex', 'info_characters_per_word',
       'info_syll_per_word', 'info_words_per_sentence',
       'info_type_token_ratio', 'info_characters', 'info_syllables',
       'info_words', 'info_wordtypes'],
      dtype='object')


In [4]:
y_train_df['stage_encoded'] = y_train_df['Coarse Label'].map({'Key Stage 2-3': 0, 'Key Stage 4-5': 1})


In [5]:
y_train_clean = y_train_df['stage_encoded']
random_state=42
zero=0
for i in y_train_clean:
    if i==0:zero+=1
print(zero)

2509


In [6]:
import re
import string
tr_text_list = x_train_df['text'].values.tolist()
def clean(text):
    text = re.sub(r'\d+','',text.lower())
    return text.translate(str.maketrans('','',string.punctuation))
for i in range(len(tr_text_list)):
    tr_text_list[i]=clean(tr_text_list[i])
bow_preprocessor = TfidfVectorizer()
X = bow_preprocessor.fit_transform(tr_text_list)

In [7]:
# def custom_preprocessor(text):
#     text = re.sub(r"_", " ", text)  # Replace underscores with spaces
#     text = re.sub(r"\d+", "", text)  # Remove numbers
#     text = re.sub(r"[^\w\s]", "", text)  # Remove punctuation
#     return text.lower().strip()  # Convert to lowercase and trim space
 
# vectorizer = TfidfVectorizer(preprocessor=custom_preprocessor)
my_bow_classifier_pipeline = sklearn.pipeline.Pipeline([
    ('bow', TfidfVectorizer(min_df=1,max_df=1, ngram_range=(1,1),stop_words=None,token_pattern = r"(?u)\b\w\w+\b",lowercase = False)),
    ('class', sklearn.linear_model.LogisticRegression(C=1.0, max_iter=200, random_state=random_state)),
])
my_parameter_grid_by_name = dict()
my_parameter_grid_by_name['bow__min_df'] = [1, 5, 10, 0.5, 0.75]
my_parameter_grid_by_name['bow__max_df'] = [0.25, 0.5, 0.75,1.0]
my_parameter_grid_by_name['bow__ngram_range'] = [(1,1),(1,2)]
my_parameter_grid_by_name['bow__stop_words'] = [None,'english']
my_parameter_grid_by_name['bow__token_pattern'] = [r'(?u)\b[\w-]+\b',r'(?u)\b\w\w+\b',r'(?u)\b(?!\d+\b)\w+\b']
my_parameter_grid_by_name['bow__lowercase'] = [False,True]
my_parameter_grid_by_name['class__C'] = np.logspace(-4, 4, 9)


#my_scoring_metric_name = 'accuracy'
my_scoring_metric_name = 'roc_auc'

In [33]:
# prng = np.random.RandomState(0)

# valid_ids = prng.choice(np.arange(N), size=round(N/5))

# valid_indicators_N = np.zeros(N)
# valid_indicators_N[valid_ids] = -1
# my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_N)
y_train_clean = list(y_train_clean)
grid_searcher = sklearn.model_selection.GridSearchCV(
    my_bow_classifier_pipeline,
    my_parameter_grid_by_name,
    scoring=my_scoring_metric_name,
    #cv=my_splitter,
    cv=sklearn.model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state),
    refit=True)
grid_searcher.fit(tr_text_list, y_train_clean)

c:\Users\ningn\anaconda3\envs\cs135_25s_env\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
7020 fits failed out of a total of 21600.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3240 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ningn\anaconda3\envs\cs135_25s_env\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\ningn\anaconda3\envs\cs135_25s_env\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ningn\anaconda3\envs\cs135_25s_env\Lib

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('bow',
                                        TfidfVectorizer(lowercase=False,
                                                        max_df=1,
                                                        token_pattern='(?u)\\b[\\w-]+\\b')),
                                       ('class',
                                        LogisticRegression(max_iter=200,
                                                           random_state=42))]),
             param_grid={'bow__lowercase': [False, True],
                         'bow__max_df': [0.25, 0.5, 0.75, 1.0],
                         'bow__min_df': [1, 5, 10, 0.5, 0.75],
                         'bow__ngram_range': [(1, 1), (1, 2)],
                         'bow__stop_words': [None, 'english'],
                         'bow__token_pattern': ['(?u)\\b[\\w-]+\\b',
                                                '(?u)\\b\\w\\w+\\b',
                                                '(?u)\\b(?!\\d+\\b)\\w+\\b'],
                         'class__C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04])},
             scoring='roc_auc')

In [ ]:
list(grid_searcher.cv_results_.keys())

['mean_fit_time',
 'std_fit_time',
 'mean_score_time',
 'std_score_time',
 'param_bow__lowercase',
 'param_bow__max_df',
 'param_bow__min_df',
 'param_bow__ngram_range',
 'param_bow__stop_words',
 'param_bow__token_pattern',
 'param_class__C',
 'params',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'mean_test_score',
 'std_test_score',
 'rank_test_score']

In [34]:
gsearch_results_df = pd.DataFrame(grid_searcher.cv_results_).copy()

param_keys = ['param_bow__min_df','param_bow__max_df','param_bow__ngram_range','param_bow__stop_words','param_bow__token_pattern','param_bow__lowercase'
              , 'param_class__C']

# Rearrange row order so it is easy to skim
gsearch_results_df.sort_values(param_keys, inplace=True)
gsearch_results_df[param_keys + ['mean_test_score', 'rank_test_score']]

,param_bow__min_df,param_bow__max_df,param_bow__ngram_range,param_bow__stop_words,param_bow__token_pattern,param_bow__lowercase,param_class__C,mean_test_score,rank_test_score
369,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.0001,NaN,2917
370,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.0010,NaN,2917
371,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.0100,NaN,2917
372,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,0.1000,NaN,2917
373,0.5,0.25,"(1, 1)",english,(?u)\b(?!\d+\b)\w+\b,False,1.0000,NaN,2917
...,...,...,...,...,...,...,...,...,...
4063,10.0,1.00,"(1, 2)",None,(?u)\b\w\w+\b,True,1.0000,0.777697,1217
4064,10.0,1.00,"(1, 2)",None,(?u)\b\w\w+\b,True,10.0000,0.773974,1323
4065,10.0,1.00,"(1, 2)",None,(?u)\b\w\w+\b,True,100.0000,0.756975,1909
4066,10.0,1.00,"(1, 2)",None,(?u)\b\w\w+\b,True,1000.0000,0.758212,1853


In [ ]:
test = gsearch_results_df[gsearch_results_df['mean_test_score'].isnull()]
test = test[test['param_bow__min_df']==1]
test = test[test['param_class__C']==100]
test = test[test['param_bow__max_df']==0.5]
test = test[test['bow__token_pattern']=="(?u)\b[\w-]+\b"]
test = test[test['param_bow__stop_words']=='english']
test = test[test['bow__ngram_range']==(1,1)]
#test = test[test['']==]
test[param_keys + ['mean_test_score', 'rank_test_score']]

KeyError: 'bow__token_pattern'

In [35]:
# Get the best-trained model (automatically refit on full training data)
best_model = grid_searcher.best_estimator_

# Best hyperparameters
best_params = grid_searcher.best_params_
print("Best Hyperparameters:", best_params)

# Best cross-validation score (average accuracy across folds)
best_score = grid_searcher.best_score_
print(f"Best CV AUROC: {best_score:.4f}")

Best Hyperparameters: {'bow__lowercase': False, 'bow__max_df': 0.5, 'bow__min_df': 1, 'bow__ngram_range': (1, 1), 'bow__stop_words': 'english', 'bow__token_pattern': '(?u)\\b[\\w-]+\\b', 'class__C': 10.0}
Best CV AUROC: 0.8281
